In [257]:
from __future__ import division

import numpy as np

from scipy.integrate import ode

import math

import matplotlib.pyplot as plt

from numpy.linalg import inv

In [258]:
no_of_iterations = 1000
#I = no_of_iterations
no_of_sampaling = 10
N = no_of_sampaling
no_of_states = 3
no_of_controls = 2
sampling_time = 0.05
Ts = sampling_time
iteration_number_index = 0
i = iteration_number_index

In [259]:
def mat_mult(a,b):
    return np.dot(a,b)

In [260]:
def lst_mult(c,d):
    return np.dot((np.array(c)),(np.array(d)))

In [261]:
def transpose_matrix(f):
    try:
        no_rows = f.shape[0]
        no_columns = f.shape[1]
    except IndexError:
        no_rows = 1
        no_columns = f.shape[0]

    u = f.T
    transpose = u.reshape(no_columns,no_rows)
    return transpose

In [119]:
#def 3D_initialization():

In [262]:
X_repo = np.zeros((no_of_iterations, no_of_sampaling, no_of_states))
U_repo = np.zeros((no_of_iterations, no_of_sampaling, no_of_controls))
x_o = y_o = 0
phi_o = 0*np.pi / 2
t_o = 0
X_repo[0,0,:] = [x_o, y_o, phi_o]
U_repo[0, :, :] = [[4,1],[0,0],[0,0],[0,0],[0,0],[6,0],[0,0],[0,0],[0,0],[0,0]]
x_star = 10.5
y_star = 10.5
phi_star = np.pi / 3
R = np.array([[1,0],[0,1]])
alpha = 0.5

jacobian_x = [[[1,1,1],[1,1,1],[1,1,1]] for itr in range(1000)]
jacobian_u = [[[1,1],[1,1],[1,1]] for itr in range(1000)]
B_o = [[[1,1,1],[1,1,1],[1,1,1]] for itr in range(1000)]
B = [[[1,1],[1,1],[1,1]] for itr in range(1000)]
#print X_repo[0,0,:]

In [263]:
def states_num_pred(t, z_states, u_controls):

        x = z_states[0]
        y = z_states[1]
        phi = z_states[2]
        
        v = u_controls[0]
        omega = u_controls[1]
        
        x_dot = v * math.cos(phi)
        y_dot = v * math.sin(phi)
        phi_dot = omega
        #j+=1
        return [x_dot, y_dot, phi_dot]

In [264]:
dy_N = np.array([100.0, 100.0, 100.0]) 
while (dy_N[0] ** 2 + dy_N[1] ** 2) > 2.9 or abs(dy_N[2]) > 0.15:

    print "iteration number", i
    sampling_inst_index = 0
    j = sampling_inst_index
    u_controls = np.array([0,0])
    integ = ode(states_num_pred)
    if i == 0:
        integ.set_initial_value(X_repo[0,0,:], t_o)
    else:
        integ.set_initial_value(X_repo[i-1, N-1, :],t_o)
    
    integ.set_integrator("dopri5")

    #integ.set_f_params(u_controls)    #for variable control inputs

    while integ.successful() and j < N-1:  #check no of iterations
        #print "printing j just before passing info in ode", j
        z_states = X_repo[i,j,:] 
        u_controls = U_repo[i,j,:]
        integ.set_f_params(u_controls)    #for variable control inputs
        s = integ.integrate(integ.t + sampling_time)
#         print"printing integrator time", integ.t
#         print "printing s", s, "for the",j,"th time"
#         print "printing sampling index",j
        j+=1
        #print "printing i in the ode solver", i
        X_repo[i,j,0] = s[0]
        X_repo[i,j,1] = s[1]
        X_repo[i,j,2] = s[2] % 6.28
        #print "printing X_repo[i, j, :] at jth sampling time", X_repo[i,j,:], j
    
    #print "X_repo is",X_repo, "at iteration number", i
    #print "printing i after ode solver", i
    dy_N[0] = x_star - X_repo[i, N -1, 0]
     
    dy_N[1] = y_star - X_repo[i, N -1, 1]
    dy_N[2] = phi_star - X_repo[i, N -1, 2]
    
    print "dy",dy_N
    #print "printing X[i,N-1,:] after RK iteration ", X_repo[i, N-1, :]
 
    dely_delx = np.identity(3)
    
    jacobian_x[N-2] = [[1,0, -(U_repo[i,N-2,0])*(math.sin(X_repo[i,N-2,2]))*Ts], [0, 1, (U_repo[i,N-2,0])*(math.cos(X_repo[i,N-2,2]))*Ts], [0, 0, Ts]]
    
    jacobian_u[N-2] = [[(math.cos(X_repo[i,N-2,2]))*Ts , 0],[(math.sin(X_repo[i,N-2,2]))*Ts , 0],[0, Ts]]
    
    B_o[N-2] = [[1,0,0],[0,1,0],[0,0,1]]
    #print "printing value of x fed into u jacobian", X_repo[i,N-2,0]
    
    b = lst_mult(B_o[N-2] , jacobian_u[N-2])
    #print "printing jacobian_u[N-2]", jacobian_u[N-2]
    B[N-2] = list(b)
    
    #B[N-2] = list(np.dot((np.array(B_o[N-2])),(np.array(jacobian_u[N-2]))))
    
    w= np.array(B[N-2])
    #print "printing B[1]",w
    x= transpose_matrix(w)
    #print "transpose of b{1}", x
    y= mat_mult(w, inv(R))
    #print "b[1]* invr",y
    z = transpose_matrix(U_repo[i, N-2,:])
    
    A_lambda = (-1) * mat_mult(y,x)
    
    b_lambda = mat_mult(w,z)
    #print "printing first b lamdbda", b_lambda
    
    for ti in range(N-3, -1,-1):
        jacobian_x[ti] = [[1,0, -(U_repo[i,ti,0])*(math.sin(X_repo[i,ti,2]))*Ts], [0, 1, (U_repo[i,ti,0])*(math.cos(X_repo[i,ti,2]))*Ts], [0, 0, Ts]]
        
        jacobian_u[ti] = [[(math.cos(X_repo[i,ti,2]))*Ts , 0],[(math.sin(X_repo[i,ti,2]))*Ts , 0],[0, Ts]]
        
        B_o[ti] = list(lst_mult(B_o[ti+1], jacobian_x[ti+1]))
        
        
        B[ti] = list(lst_mult(B_o[ti], jacobian_u[ti]))

        
        o= np.array(B[ti])
        
        p= transpose_matrix(o)
        q= mat_mult(o, inv(R))
        r = transpose_matrix(U_repo[i, ti,:])
        
        A_lambda += (-1) * mat_mult(q,p)
        b_lambda += mat_mult(o,r)
        #print "printing b lambda in loop ", b_lambda, "for ith iteration and jth time inst", i, ti
        #print "Alambda",A_lambda
        #print "dy",dy_N
        
    i+=1
    
    for k in range(N-1):
        g = (1 - alpha) * (transpose_matrix(U_repo[i-1, k, :]))
        l = mat_mult(inv(R), transpose_matrix(np.array(B[k])))
        m = mat_mult(l , inv(A_lambda))
        mn = transpose_matrix(dy_N)
        n = mat_mult(m, (mn - b_lambda))
        
        c = g + (alpha)* n
        #print "printing blambda",b_lambda
#         print "printing dYn",dy_N
        #print "printing dy - blmbda",mn - b_lambda
#         print "printing m matrix",m
#         print "printing l matrix",l
#         print "printing c matrix",c
#         print "printing g matrix",g
#         print "printing n matrix",n 
        U_repo[i,k,:] = transpose_matrix(c)
    if i == 999:
        print "reached last iteration without convergnce and the U_repo is:", U_repo , X_repo
        break
        #print "printing U repo after storing next iteration values", U_repo
        
       
        

        

iteration number 0
dy [ 10.76950144  10.28104646   0.99719755]
iteration number 1
dy [ 10.5         10.5          1.04719755]
iteration number 2
dy [ 10.5         10.5          1.04719755]
iteration number 3
dy [ 10.37023042  10.49100852  -0.15003522]
iteration number 4
dy [ 10.09060761   9.86011096  -4.56990305]
iteration number 5
dy [ 10.30926611   9.68774565  -3.24704754]
iteration number 6
dy [ 10.23129962   9.65088559  -1.77036132]
iteration number 7
dy [ 10.10938123   9.51759013  -2.28437509]
iteration number 8
dy [ 10.38565037   9.20024585  -2.52106129]
iteration number 9
dy [ 10.59084355   9.25956603  -4.30179782]
iteration number 10
dy [ 10.50607416   9.81032178  -0.40735098]
iteration number 11
dy [ 10.54597264   9.78122469  -1.33723234]
iteration number 12
dy [ 10.49823643   9.81121217  -1.12094738]
iteration number 13
dy [ 10.50613279   9.76651308  -0.93103771]
iteration number 14
dy [ 10.96363991   9.3435629   -1.24669068]
iteration number 15
dy [ 11.03334746  11.89444975 